In [12]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### Here I read my predicted file, but you just have to make your own predicted Pandas DataFrame having the same  column names and format as mine.

In [7]:
predict_file = pd.read_csv("NeuralNet_pred_T.csv", usecols = [1,2,3,4])

In [8]:
predict_file[:5]

,username,anime_id,predicted_score,true_score
0,Getsa,7647,5.028807,7
1,Rummy,656,7.416133,8
2,Btooomz,6213,9.076716,11
3,arashi176,2418,8.288404,10
4,DraigGoch,19369,7.081574,8


In [55]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for i in tqdm(range(len(predictions))):
        top_n[predictions.username.iloc[i]].append((predictions["true_score"].iloc[i], predictions["predicted_score"].iloc[i]))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [67]:
def get_top_n_anime(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for i in tqdm(range(len(predictions))):
        top_n[predictions.username.iloc[i]].append([predictions["true_score"].iloc[i], 
                                                    predictions["predicted_score"].iloc[i],
                                                    predictions["anime_id"].iloc[i]])

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        TrueTopNAnime = [i[2] for i in user_ratings[:n]]
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        PredictedTopNAnime = [i[2] for i in user_ratings[:n]]
        
        top_n[uid] = {"true": TrueTopNAnime, "predicted": PredictedTopNAnime}

    return top_n

In [34]:
def ndcg(y_true, y_pred, k=None, powered=False):
    def dcg(scores, k=None, powered=False):
        if k is None or k > scores.shape[0]:
            k = scores.shape[0]
        if not powered:
            ret = scores[0]
            for i in range(1, k):
                ret += scores[i] / np.log2(i + 1)
            return ret
        else:
            ret = 0
            for i in range(k):
                ret += (2 ** scores[i] - 1) / np.log2(i + 2)
            return ret
    
    ideal_sorted_scores = np.sort(y_true)[::-1]
    ideal_dcg_score = dcg(ideal_sorted_scores, k=k, powered=powered)
    
    pred_sorted_ind = np.argsort(y_pred)[::-1]
    pred_sorted_scores = y_true[pred_sorted_ind]
    dcg_score = dcg(pred_sorted_scores, k=k, powered=powered)
    
    return dcg_score / ideal_dcg_score

def ndcg1(y_true, y_pred, k=None):
    return ndcg(y_true, y_pred, k=k, powered=False)

def ndcg2(y_true, y_pred, k=None):
    return ndcg(y_true, y_pred, k=k, powered=True)

In [76]:
def MAP_k(y_true, y_pred, k = None):
    if k is None or k > len(y_true):
        k = len(y_true)
    intersect_count = 0
    for i in y_pred:
        if i in y_true:
            intersect_count +=1
    AP = intersect_count/len(y_true)
    return AP

In [45]:
set(list([1,1,2,3]))

{1, 2, 3}

In [57]:
top_n = get_top_n(predict_file, n=10)
#print(top_n)
users_est = defaultdict(list)
users_true=defaultdict(list)
for uid, user_ratings in top_n.items():
    users_est[uid].append([est for (_, est) in user_ratings])
    users_true[uid].append([true_r for (true_r, _) in user_ratings])

In [58]:
ndcg_list=[]
for uid in top_n:
    
    for i in users_true[uid]:
        y_true=np.asarray(i)#.reshape(-1,1)
    for i in users_est[uid]:
        y_pred=np.asarray(i)#.reshape(-1,1)
    
        ndcg_list.append(ndcg1(y_true, y_pred, k=None))

In [59]:
ndcg_list[:5]

[0.9135188536761149,
 0.9541773930368265,
 0.9746549955902717,
 0.735386108273794,
 0.9412789242613623]

In [60]:
ndcg_list = [i for i in ndcg_list if str(i) != 'nan']
np.mean(ndcg_list)

0.8933941456367094

In [63]:
del top_n

In [68]:
TopNAnime = get_top_n_anime(predict_file, 10)

In [77]:
MAP_list=[]
for uid in TopNAnime:
    MAP_list.append( 
                    MAP_k(TopNAnime[uid]["true"], 
                          TopNAnime[uid]["predicted"], 
                          k=10))



In [80]:
MAP_list[:10]

[0.3, 0.3, 0.3, 0.3, 0.3, 0.4, 0.2, 0.3, 0.3, 0.6]

In [79]:

MAP_list = [i for i in MAP_list if str(i) != 'nan']
np.mean(MAP_list)

0.4995707629312373